In [1]:
from keras.applications import VGG16

# VGG16 was designed to work on 224x224 pixel images
img_rows, img_cols = 224, 224

# loading the VGG16 without the top
vgg16 = VGG16(weights='imagenet', 
              include_top=False, 
              input_shape=(img_rows, img_cols, 3))

Using TensorFlow backend.






58892288/58889256 [==============================] - 5s 0us/step








In [2]:
# checking out all the layers
for (i, layer) in enumerate(vgg16.layers):
    print(str(i) + '. ' + layer.__class__.__name__ + ' - ' + str(layer.trainable))

0. InputLayer - False
1. Conv2D - True
2. Conv2D - True
3. MaxPooling2D - True
4. Conv2D - True
5. Conv2D - True
6. MaxPooling2D - True
7. Conv2D - True
8. Conv2D - True
9. Conv2D - True
10. MaxPooling2D - True
11. Conv2D - True
12. Conv2D - True
13. Conv2D - True
14. MaxPooling2D - True
15. Conv2D - True
16. Conv2D - True
17. Conv2D - True
18. MaxPooling2D - True


In [3]:
for layer in vgg16.layers:
    layer.trainable = False

for (i, layer) in enumerate(vgg16.layers):
    print(str(i) + '. ' + layer.__class__.__name__ + ' - ' + str(layer.trainable))

0. InputLayer - False
1. Conv2D - False
2. Conv2D - False
3. MaxPooling2D - False
4. Conv2D - False
5. Conv2D - False
6. MaxPooling2D - False
7. Conv2D - False
8. Conv2D - False
9. Conv2D - False
10. MaxPooling2D - False
11. Conv2D - False
12. Conv2D - False
13. Conv2D - False
14. MaxPooling2D - False
15. Conv2D - False
16. Conv2D - False
17. Conv2D - False
18. MaxPooling2D - False


In [0]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization 
from keras.models import Model

In [0]:
def addTopModel(bottom_model, num_classes, D=256):
    
    top_model = bottom_model.output
    top_model = Flatten(name='flatten')(top_model)
    top_model = Dense(D, activation='relu')(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation='softmax')(top_model)
    
    return top_model

In [6]:
num_classes = 17

FC_Head = addTopModel(vgg16, num_classes)
model = Model(inputs=vgg16.input, output=FC_Head)

print(model.summary())


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  after removing the cwd from sys.path.


In [7]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = '/content/drive/My Drive/Colab Notebooks/Faaltu/CNN/6. Fine Tuning and Transfer Learning/flower-dataset/train'
validation_data_dir = '/content/drive/My Drive/Colab Notebooks/Faaltu/CNN/6. Fine Tuning and Transfer Learning/flower-dataset/validation'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(
    rescale=1./255)

train_batch_size = 16
val_batch_size = 10

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_rows, img_cols),
    batch_size=train_batch_size,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_rows, img_cols),
    batch_size=val_batch_size,
    class_mode='categorical',
    shuffle=False)

Found 1200 images belonging to 17 classes.
Found 170 images belonging to 17 classes.


In [11]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

chkp_save_path = '/content/drive/My Drive/Colab Notebooks/Faaltu/CNN/6. Fine Tuning and Transfer Learning/saved-models/flower_classifier.h5'

checkpoint = ModelCheckpoint(chkp_save_path,
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystopping = EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=3,
                              verbose=1,
                              restore_best_weights=True)

callbacks = [checkpoint, earlystopping]

model.compile(optimizer=RMSprop(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

nb_train_samples = 1190
nb_val_samples = 170
epochs = 25
batch_size = 16

history = model.fit_generator(
    train_generator,
    steps_per_epoch= nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_val_samples // batch_size)

model.save('/content/drive/My Drive/Colab Notebooks/Faaltu/CNN/6. Fine Tuning and Transfer Learning/saved-models/flower_classifier.h5')

Epoch 1/25
74/74 [==============================] - 27s 361ms/step - loss: 0.9072 - acc: 0.7120 - val_loss: 0.6489 - val_acc: 0.8200

Epoch 00001: val_loss improved from inf to 0.64894, saving model to /content/drive/My Drive/Colab Notebooks/Faaltu/CNN/6. Fine Tuning and Transfer Learning/saved-models/flower_classifier.h5
Epoch 2/25
74/74 [==============================] - 26s 350ms/step - loss: 0.7854 - acc: 0.7534 - val_loss: 0.6475 - val_acc: 0.8700

Epoch 00002: val_loss improved from 0.64894 to 0.64753, saving model to /content/drive/My Drive/Colab Notebooks/Faaltu/CNN/6. Fine Tuning and Transfer Learning/saved-models/flower_classifier.h5
Epoch 3/25
74/74 [==============================] - 25s 339ms/step - loss: 0.6961 - acc: 0.7838 - val_loss: 0.5810 - val_acc: 0.8800

Epoch 00003: val_loss improved from 0.64753 to 0.58102, saving model to /content/drive/My Drive/Colab Notebooks/Faaltu/CNN/6. Fine Tuning and Transfer Learning/saved-models/flower_classifier.h5
Epoch 4/25
74/74 [==